In [16]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline
from pythainlp.tokenize import word_tokenize

model_path = 'checkpoint-epoch-50'

tokenizer = AutoTokenizer.from_pretrained(model_path,  model_max_length=512)
model = AutoModelForTokenClassification.from_pretrained(model_path)
pipe = TokenClassificationPipeline(model=model, tokenizer=tokenizer)


In [17]:
def predict(text):
    # word tokenize
    text_token = word_tokenize(text)
    text_token = ' '.join(text_token)
    prediction = pipe(text_token, grouped_entities=True, ignore_labels=[])
    # covert predict to html tag
    text_pred = ""
    for dict_pred in prediction:
        open_tag = f"<{dict_pred['entity_group'].lower()}>"
        close_tag = f"</{dict_pred['entity_group'].lower()}>"
        group_word = dict_pred['word']

        if group_word.strip() == "":
            text_pred += group_word
        elif open_tag == "<o>":
            text_pred += ''.join(group_word.split(" "))
        else:
            group_word = ''.join(group_word.split(" "))
            text_pred += open_tag + group_word + close_tag
    return text_pred

In [18]:

text = '''ปกติถ้าสีข้าวเปลือกเป็นข้าวขาว โรงสีจะได้กำไรจาก Byproduct เอาไปขายต่อได้คือ แกลบ รำ ปลายข้าว
แต่ถ้าสีข้าวปลือกเป็นข้าวกล้อง โรงสีจะเหลือ Byproduct แค่ แกลบ อย่างเดียว
ทำให้ข้าวกล้องแพงกว่าข้างขาวเพราะโรงสีต้องบวกกำไรที่เคยได้จาก Byproduct ของข้าวขาวเพิ่มไปในราคาข้าวกล้อง'''

predict(text)

'<claim>ปกติถ้าสีข้าวเปลือกเป็น</claim>ข้าวขาวโรงสี<premist>จะได้กําไรจากByproductเอาไปขายต่อได้คือแกลบรําปลายข้าว</premist>แต่ถ้าสีข้าวปลือกเป็นข้าวกล้องโรงสีจะเหลือByproductแค่แกลบอย่าง<claim>เดียวทําให้ข้าวกล้องแพงกว่าข้างขาว</claim><premist>เพราะโรงสีต้องบวกกําไรที่เคยได้จากByproductของข้าวขาวเพิ่มไปในราคาข้าวกล้อง</premist>'